# Predict Contracts for ALL 2024 Players

In [1]:
import pickle
import pandas as pd

## Load Best Model

In [2]:
with open('./models/best_model.pkl', 'rb') as f:
    forest = pickle.load(f)
    
with open('./models/best_model_3rd.pkl', 'rb') as f:
    forest_3rd = pickle.load(f)

## Build the 2024 Player Stats Dataset

### Load Data

In [3]:
player_stats = pd.read_csv('./raw_data/2024_player_stats.csv')
advanced = pd.read_csv('./advanced_stats/players_advanced_2023.csv')

### Transform Data

In [4]:
advanced = advanced.drop(columns = ['Unnamed: 19', 'Unnamed: 24'])

# Get Player Stats + Advanced Stats of 2024
player_stats['Player-additional'] = player_stats['Player-additional'].astype(str)
advanced['Player-additional'] = advanced['Player-additional'].astype(str)

# Merge together
full_2024_stats = player_stats.set_index('Player-additional').join(advanced.set_index('Player-additional'), lsuffix='0', how='inner')

# Remove Columns
full_2024_stats = full_2024_stats[['Player0', 'Pos0', 'Age0', 'eFG%', 'TRB', 'PTS', 'AST', 'STL', 'BLK',
               'WS', 'BPM', 'VORP']]

# Create New Features
full_2024_stats = full_2024_stats.reset_index(drop=True).rename(columns={'Player0': 'Player',
                                                       'Pos0': 'POS',
                                                       'Age0': 'AGE',
                                                       'eFG%': 'eFG'})
full_2024_stats['FA_Year'] = 2023
full_2024_stats = full_2024_stats.dropna()
full_2024_stats['POS_SF'] = pd.get_dummies(full_2024_stats['POS'])['SF']
full_2024_stats['POS_PF'] = pd.get_dummies(full_2024_stats['POS'])['PF']

## Create the 2024 Testing Dataset

In [5]:
X_test = full_2024_stats.drop(columns = ['Player', 'POS'])[['FA_Year', 'AGE', 'eFG', 'PTS', 'AST', 'TRB', 'STL', 'BLK', 'WS', 'BPM', 
        'VORP', 'POS_PF', 'POS_SF']]
X_test = X_test.dropna()

## Predict the Salaries of 2024 Players on the Trained RF Model

In [6]:
y_preds = forest.predict(X_test) # normal predictions
y_3rd = forest_3rd.predict(X_test, quantiles = [0.75]) # upper bound predictions

/Users/joshuali/opt/anaconda3/envs/PIC16B/lib/python3.8/site-packages/sklearn/base.py:402: UserWarning: X has feature names, but RandomForestQuantileRegressor was fitted without feature names
  warnings.warn(


In [7]:
# add to the DF
full_2024_stats['EXPECTED_CONT'] = y_preds
full_2024_stats['UPPER_LIMIT'] = y_3rd

In [8]:
stats_and_preds = full_2024_stats.sort_values(by='EXPECTED_CONT', ascending = False)

### Miscellaneous Predictions for Boston Celtics Players with Contract Extensions

In [9]:
stats_and_preds.loc[stats_and_preds['Player'] == 'Derrick White', :]

,Player,POS,AGE,eFG,TRB,PTS,AST,STL,BLK,WS,BPM,VORP,FA_Year,POS_SF,POS_PF,EXPECTED_CONT,UPPER_LIMIT
540,Derrick White,SG,29,0.578,4.2,15.2,5.2,1.0,1.2,8.5,3.8,3.5,2023,0,0,2.568292e+07,32124579.5


In [10]:
stats_and_preds.loc[stats_and_preds['Player'] == 'Al Horford', :]

,Player,POS,AGE,eFG,TRB,PTS,AST,STL,BLK,WS,BPM,VORP,FA_Year,POS_SF,POS_PF,EXPECTED_CONT,UPPER_LIMIT
226,Al Horford,C,37,0.64,6.4,8.6,2.6,0.6,1.0,6.2,3.6,2.5,2023,0,0,9.544650e+06,1.147728e+07


In [11]:
stats_and_preds.loc[stats_and_preds['Player'].str.contains('Kristaps'), :]

,Player,POS,AGE,eFG,TRB,PTS,AST,STL,BLK,WS,BPM,VORP,FA_Year,POS_SF,POS_PF,EXPECTED_CONT,UPPER_LIMIT
408,Kristaps Porziņģis,C,28,0.589,7.2,20.1,2.0,0.7,1.9,7.9,5.1,3.0,2023,0,0,2.606640e+07,31600000.0


## Write to Output

In [12]:
stats_and_preds.to_csv("./output/2024_preds.csv", index=False)